# Lab 2: Publish & Subscribe

Table of contents

1. [Overview](#overview)
    1. [Features](#features)
1. [Prerequisites](#prerequisites)
1. [Common pubsub operations](#common-pubsub-operations)
    1. [Subscribing to a topic](#subscribing-to-a-topic)
    1. [Publishing message to a topic](#publishing-message-to-a-topic)
1. [Clean up](#clean-up)
1. [Running the quickstart](#running-the-quickstart)
1. [Next steps](#next-steps)
    1. [References](#references)

## Overview

Dapr provides publish & subscribe APIs for microservices to communicate with each other using messages for event-driven architectures. You can use Dapr to publish message on a topic to a supported message broker ( such as Kafka, Redis etc.) and also subscribe to topics. The message broker will receive messages for a topic from the publisher and send it to all the subscribers of the topic. The list of brokers supported by Dapr can be found [here](https://docs.dapr.io/reference/components-reference/supported-pubsub/). 

In the diagram below, the Dapr API posts an `order` topic via the "publishing" cart service, which is received by the "order" endpoints on the “shipping” and “email” subscribing services.

<img src="../static/02-pubsub-API.png">

In this lab, you will learn how to use the Dapr APIs to publish and subscribe.

### Features

Dapr pub/sub API provides several features to your application. Some of these are as follows:

1. At-least-once guarantee
1. Sending messages using Cloud Events
1. Setting message content types
1. Communication with applications not using Dapr and CloudEvents
1. Handling failed messages with dead letter topics
1. Message Time-to-Live (TTL)

## Prerequisites

1. [Install Dapr CLI](https://docs.dapr.io/getting-started/install-dapr-cli/).
1. Initialize Dapr in your local environment, run `dapr init` in your terminal.
1. Run the cell below to import some helper functions used in this lab.

In [ ]:
# Run this cell to import the necessary libraries
import json
import os
import sys
import time

sys.path.append(os.path.abspath('../utils'))
from shell import execute, execute_async

If you have set up everything correctly, the following command should display:
```text
CLI version: x.y.z 
Runtime version: a.b.c
```

In [ ]:
print(execute("dapr --version"))

## Common pubsub operations

Now that the setup is done, we will be using Dapr for subscribing as well as publishing messages. Note that the messaging broker used below is `Redis` which is by default installed during `dapr init`. 

### Subscribing to a topic

For the subscriber, we will be using `order-processor` app from quickstarts. It subscribes to `orders` topic on Redis instance `orderpubsub` (defined in `pubsub.yaml` in components folder) with `orders` registered as the callback route. All published messages on the subscribed topic would be delivered on this route via Dapr sidecar.


```py
# Register Dapr pub/sub subscriptions
@app.route('/dapr/subscribe', methods=['GET'])
def subscribe():
    subscriptions = [{
        'pubsubname': 'orderpubsub',
        'topic': 'orders',
        'route': 'orders'
    }]
    print('Dapr pub/sub is subscribed to: ' + json.dumps(subscriptions))
    return jsonify(subscriptions)


# Dapr subscription in /dapr/subscribe sets up this route
@app.route('/orders', methods=['POST'])
def orders_subscriber():
    event = from_http(request.headers, request.get_data())
    print('Subscriber received : ' + event.data['orderid'], flush=True)
    return json.dumps({'success': True}), 200, {
        'ContentType': 'application/json'}


app.run(port=5001)
```




For running subscriber app, first clone the quickstarts repo.
```bash
git clone https://github.com/dapr/quickstarts.git
```
Navigate to the `order-processor` directory, which contains the subscriber application.
```bash
cd quickstarts/pub_sub/python/http/order-processor
```
Install the dependencies.
```bash
pip3 install -r requirements.txt
```
Finally run the subscriber application alonside Dapr sidecar.
```bash
dapr run --app-id order-processor --components-path ../../../components/ --app-port 5001 --dapr-http-port 3501 -- python3 app.py
```

### Publishing messages to a topic

For publishing message, first we need to run a dapr sidecar with same pubsub component as subscriber. Publisher application will send a POST request to dapr sidecar on `publish` endpoint and Dapr in-turn will publish messages to the broker.

Run the cell below to start the sidecar.

In [ ]:
OrderProcessorDirectory = "quickstarts/pub_sub/python/http/order-processor"
execute_async(f"cd {OrderProcessorDirectory} && dapr run --app-id checkout --dapr-http-port 3500 --components-path ../../../components/")

Validate that Dapr is running!

Expected output:
```text
  APP ID        HTTP PORT  GRPC PORT  APP PORT  COMMAND  AGE  CREATED  DAPRD PID  CLI PID  
  checkout        3500      <random>      0               <X> <datetime>  <pid>     <pid>  
  order-processor 3501      <random>     5001             <X> <datetime>  <pid>     <pid>  
```

In [ ]:
print(execute("dapr list"))

Run the cell below to publish a json object `{"orderId":101}` to `orders` topic. We are using `curL` here to send the request.

In [ ]:
request_body = json.dumps({"orderId":101})
print(request_body)
execute(f'curl -X POST \
    -H "Content-Type: application/json" \
    -d \'{request_body}\' \
    http://localhost:3500/v1.0/publish/orderpubsub/orders')

You should see the below log in the terminal running subscriber application.

```text
== APP == Subscriber received : 101
```

Dapr CLI can also be used for publishing message. Run cell below to publish another message on the same topic.

In [ ]:
request_body = json.dumps({"orderId":102})
execute(f'dapr publish --publish-app-id checkout --pubsub orderpubsub --topic orders --data \'{request_body}\'')

Again you should see the below log in the terminal running subscriber application.

```text
== APP == Subscriber received : 102
```

## Clean up

Stop the Dapr process that we started earlier for both publisher and Subscriber. Expected output:
```text
✅  app stopped successfully: checkout
✅  app stopped successfully: order-processor
```

In [ ]:
print(execute("dapr stop --app-id checkout"))
print(execute("dapr stop --app-id order-processor"))

## Running the quickstart

Next, check out the quickstart from https://github.com/dapr/quickstarts/tree/master/pub_sub. There are multiple programming languages available, you can choose the one of your choice.

Within each quickstart, there are two flavors available
1. HTTP - Uses a simple HTTP client to invoke Dapr APIs (similar to what we did in this lab).
1. SDK - Uses special Dapr SDKs to call Dapr APIs. This makes it easier to write Dapr applications, as the SDKs abstract away the HTTP calls.

You have already seen how to make HTTP calls to invoke Dapr, so you can try out quickstarts using the SDKs. If you are interested in learning more about the Dapr SDKs, you can read more about them [here](https://docs.dapr.io/developing-applications/sdks).



To run the quickstart, you can follow the instructions from here https://github.com/dapr/quickstarts/tree/master/pub_sub/python/sdk

## Next steps

Thank you for completing this lab! Please check out the references below to learn more.

### References

1. Publish Subscribe overview: https://docs.dapr.io/developing-applications/building-blocks/pubsub/pubsub-overview/
1. Publish Subscribe API reference: https://docs.dapr.io/reference/api/pubsub_api/
1. Publish Subscribe quickstarts: https://github.com/dapr/quickstarts/tree/master/pub_sub